In [1]:
import numpy as np

import pandas as pd

In [2]:
df = pd.read_csv("summer.csv")
df.head()

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver


In [6]:
len(df.Sport.unique())

43

In [5]:
len(df.Discipline.unique())

67

In [80]:
# df[['City', 'Sport', 'Discipline']].value_counts().index.values\

In [68]:
# SELECT A 
# FROM R 
# GROUP BY A 
# HAVING COUNT (DISTINCT B) > 1; 

for group in df.groupby(['Event', 'City'])['Sport']:
    if len(group[1].unique()) > 1:
        print(group[1])
        print("^ Violation of FD rule")
        break


820        Boxing
821        Boxing
822        Boxing
1115    Wrestling
1116    Wrestling
1117    Wrestling
Name: Sport, dtype: object
^ Violation of FD rule


In [4]:
set(df.columns)

{'Athlete',
 'City',
 'Country',
 'Discipline',
 'Event',
 'Gender',
 'Medal',
 'Sport',
 'Year'}

In [8]:
from itertools import chain, combinations


def powerset(iterable):
    # https://stackoverflow.com/questions/1482308/how-to-get-all-subsets-of-a-set-powerset
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

        


In [74]:
def check_fd(A_set, B, df):
    # If two tuples of R agree on all of the attributes A 1 ; A2 ;::: ;An 
    # (i.e., the tuples have the same values in their respective components for each of these attributes),  
    # then they must also agree on all of another list of attributes B 1 ; B2 ;::: ;Bm
    
    # For all values of A_set
    #    check if given a value of A_set, the B is same
    

    
    for group in df.groupby([*A_set])[B]:
        if len(group[1].unique()) > 1:

            return False
        
    return True
    

In [71]:
def get_all_candidates(df):
    candidate_list = []
    for tup in list(powerset(set(df.columns))):
        if set(tup) not in candidate_list and len(set(tup)) > 0:
            candidate_list.append(set(tup))
    return candidate_list

# print(get_all_candidates(df))

In [78]:
def get_all_fds(df):
    # Rational for just checking against singleton set
    # Bs: "It is common for the right side of an FD to be a single attribute. 
    # In fact,  we shall see that the one functional dependency A 1 A 2    A n -> B 1 B 2    B m 
    # is  equivalent to the set of FD's: 
    # A 1 A 2    A n -> B 1  
    # A 1 A 2    A n -> B 2  
    #    
    # A 1 A 2    A n -> B"
    # (Database Systems the Complete Book p. 68)
    fds = []
    a_candidates = get_all_candidates(df)
    b_candidates = set(df.columns)
    print("required iterations = {val}".format(val=len(a_candidates)*len(b_candidates)))
    i = 0
    for a_candidate in a_candidates:
        for b_candidate in b_candidates:
            i += 1
            if i % 100 == 0:
                print(i/len(a_candidates)*len(b_candidates), "% complete")
            if b_candidate not in a_candidate: # this check to make sure non-trivial
                if check_fd(a_candidate, b_candidate, df):
                    fds.append((a_candidate, b_candidate))
    return fds

In [79]:
olympic_fds = get_all_fds(df)
print(olympic_fds)

required iterations = 4599
1.761252446183953 % complete
3.522504892367906 % complete
5.283757338551859 % complete
7.045009784735812 % complete
8.806262230919765 % complete
10.567514677103718 % complete
12.32876712328767 % complete
14.090019569471623 % complete
15.851272015655576 % complete
17.61252446183953 % complete
19.373776908023483 % complete
21.135029354207436 % complete
22.89628180039139 % complete
24.65753424657534 % complete
26.418786692759298 % complete
28.180039138943247 % complete
29.941291585127203 % complete
31.702544031311152 % complete
33.46379647749511 % complete
35.22504892367906 % complete
36.986301369863014 % complete
38.74755381604697 % complete
40.50880626223092 % complete
42.27005870841487 % complete
44.03131115459883 % complete
45.79256360078278 % complete
47.55381604696673 % complete
49.31506849315068 % complete
51.07632093933464 % complete
52.837573385518596 % complete
54.59882583170254 % complete
56.360078277886494 % complete
58.121330724070454 % complete
59.

In [334]:
#list([set(x) for x in list(powerset(set(df.columns))) if len(x) > 0])

In [115]:
from closure import *

### Now it is time to find the BCNF decomposition

In [368]:
global_R_attrs = set(df.columns)

def check_superkey(a_set, FDs):
    implied = set()
    for fd in FDs:
        for a_combo in list([set(x) for x in list(powerset(a_set)) if len(x) > 0]):
            if a_combo == fd[0]:
                implied.add(fd[1])
    
    if implied == global_R_attrs:
        return True
    else:
        return False

# def compute_closure(x, FDs): 
#     # Return the closure of x but not including x
#     pass

memo = {}
def bcnf_decompose(R_attrs, verbose=False):
    # A relation R is in BCNF if and only if: 
    # whenever there is a nontrivial FD  A1 A2 ... An -> B1 B2 ... Bm for R,
    # it is the case that { A1, A2 , ... , An } is  a superkey for R

    attr_combos = list([set(x) for x in list(powerset(R_attrs)) if len(x) > 0])
    
    for attr_combo in attr_combos:
        
        x_closure = compute_closure(attr_combo, reversed(olympic_fds)).intersection(R_attrs)
        if not is_superkey(attr_combo, R_attrs, reversed(olympic_fds)) and x_closure != attr_combo:
            
            print(attr_combo, "bad fd", x_closure)
            
            if verbose:
                print("R_attrs:", R_attrs)
                print("X or attr_combo:", attr_combo)
         
                
            z = R_attrs - x_closure 
           
            y = x_closure - attr_combo
            
            if verbose:
                print("y :", y)
                print("z: ", z)
            
            r1 = bcnf_decompose(y.union(attr_combo))
            r2 = bcnf_decompose(z.union(attr_combo))
            
            return [r1, r2]
        
    return R_attrs
    
    
    
def bcnf(R_attrs, verbose=False):
    return bcnf_decompose(R_attrs, verbose)



In [382]:
decomp = bcnf(global_R_attrs, reversed(olympic_fds)) 
decomp1 = bcnf(global_R_attrs, olympic_fds)

{'Discipline'} bad fd {'Discipline', 'Sport'}
R_attrs: {'Discipline', 'Gender', 'Country', 'Athlete', 'Year', 'Medal', 'Sport', 'Event', 'City'}
X or attr_combo: {'Discipline'}
y : {'Sport'}
z:  {'Gender', 'Country', 'Medal', 'Year', 'Athlete', 'Event', 'City'}
{'Year'} bad fd {'Year', 'City'}
{'Event', 'Athlete'} bad fd {'Event', 'Gender', 'Athlete'}
{'Year', 'Event', 'Athlete'} bad fd {'Year', 'Event', 'Country', 'Athlete'}
{'Discipline'} bad fd {'Discipline', 'Sport'}
R_attrs: {'Discipline', 'Gender', 'Country', 'Athlete', 'Year', 'Medal', 'Sport', 'Event', 'City'}
X or attr_combo: {'Discipline'}
y : {'Sport'}
z:  {'Gender', 'Country', 'Medal', 'Year', 'Athlete', 'Event', 'City'}
{'Year'} bad fd {'Year', 'City'}
{'Event', 'Athlete'} bad fd {'Event', 'Gender', 'Athlete'}
{'Year', 'Event', 'Athlete'} bad fd {'Year', 'Event', 'Country', 'Athlete'}


In [384]:
decomp

[{'Discipline', 'Sport'},
 [{'City', 'Year'},
  [{'Athlete', 'Event', 'Gender'},
   [{'Athlete', 'Country', 'Event', 'Year'},
    {'Athlete', 'Discipline', 'Event', 'Medal', 'Year'}]]]]

In [9]:
len(df.Year.unique())

27

In [10]:
len(df.City.unique())

22

In [11]:
len(df.Event.unique())

666

In [15]:
len(df.Discipline.unique())

67

In [16]:
len(df.Sport.unique())

43

In [14]:
df[df.Sport=='Gymnastics']

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
72,1896,Athens,Gymnastics,Artistic G.,"WEINGÄRTNER, Hermann",GER,Men,Horizontal Bar,Gold
73,1896,Athens,Gymnastics,Artistic G.,"FLATOW, Alfred",GER,Men,Horizontal Bar,Silver
74,1896,Athens,Gymnastics,Artistic G.,"FLATOW, Alfred",GER,Men,Parallel Bars,Gold
75,1896,Athens,Gymnastics,Artistic G.,"ZUTTER, Louis",SUI,Men,Parallel Bars,Silver
76,1896,Athens,Gymnastics,Artistic G.,"ZUTTER, Louis",SUI,Men,Pommel Horse,Gold
...,...,...,...,...,...,...,...,...,...
30390,2012,London,Gymnastics,Trampoline,"USHAKOV, Dmitry",RUS,Men,Individual,Silver
30391,2012,London,Gymnastics,Trampoline,"LU, Chunlong",CHN,Men,Individual,Bronze
30392,2012,London,Gymnastics,Trampoline,"MACLENNAN, Rosannagh",CAN,Women,Individual,Gold
30393,2012,London,Gymnastics,Trampoline,"HUANG, Shanshan",CHN,Women,Individual,Silver


In [385]:
for group in df.groupby(['City'])['Year']:
        if len(group[1].unique()) > 1:
            print(group)

('Athens', 0        1896
1        1896
2        1896
3        1896
4        1896
         ... 
27169    2004
27170    2004
27171    2004
27172    2004
27173    2004
Name: Year, Length: 2149, dtype: int64)
('London', 1133     1908
1134     1908
1135     1908
1136     1908
1137     1908
         ... 
31160    2012
31161    2012
31162    2012
31163    2012
31164    2012
Name: Year, Length: 3567, dtype: int64)
('Los Angeles', 5714     1932
5715     1932
5716     1932
5717     1932
5718     1932
         ... 
18046    1984
18047    1984
18048    1984
18049    1984
18050    1984
Name: Year, Length: 2074, dtype: int64)
('Paris', 151     1900
152     1900
153     1900
154     1900
155     1900
        ... 
4999    1924
5000    1924
5001    1924
5002    1924
5003    1924
Name: Year, Length: 1396, dtype: int64)
